In [1]:
#!pip install pandas
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [2]:
#grab data
data_path = 'data21.csv'
data = pd.read_csv(data_path)
#print
data.describe()

,m1 h,m2 h,m3 h,m4 h,m1 a,m2 a,m3 a,m4 a,winner
count,320.000000,320.000000,320.000000,320.000000,320.000000,320.000000,320.000000,320.000000,320.000000
mean,0.275938,-0.022750,-0.262042,-1.256399,0.315563,0.017219,-0.273075,-0.030644,0.565625
std,1.715917,0.839123,1.187432,20.028346,1.437422,0.830334,1.169269,2.128018,0.496451
min,-15.990000,-1.550000,-8.627143,-355.150000,-8.530000,-1.620000,-6.585366,-20.881967,0.000000
25%,-0.372500,-0.680000,-0.801402,-0.883522,-0.407500,-0.582500,-0.898254,-0.865959,0.000000
50%,0.415000,-0.050000,-0.054243,0.213582,0.320000,-0.030000,-0.031143,0.234643,1.000000
75%,1.202500,0.600000,0.400611,1.167263,1.230000,0.520000,0.382997,1.184935,1.000000
max,4.130000,1.820000,3.080944,3.830000,4.940000,1.920000,2.661574,5.530000,1.000000


In [35]:
#y is the value we are trying to predict
y = data.winner
#these are the variables we are using to make the prediction
features = ['m1 h', 'm2 h', 'm3 h', 'm4 h', 'm1 a', 'm2 a', 'm3 a', 'm4 a']
x = data[features]

corr = x.corr()
corr.style.background_gradient(cmap='copper').format(precision=2)

,m1 h,m2 h,m3 h,m4 h,m1 a,m2 a,m3 a,m4 a
m1 h,1.00,0.08,-0.19,0.08,0.06,0.07,-0.05,0.02
m2 h,0.08,1.00,0.07,0.07,0.00,-0.17,0.03,-0.02
m3 h,-0.19,0.07,1.00,0.03,-0.02,-0.04,0.01,0.00
m4 h,0.08,0.07,0.03,1.00,-0.00,-0.02,0.02,0.01
m1 a,0.06,0.00,-0.02,-0.00,1.00,0.05,-0.19,0.52
m2 a,0.07,-0.17,-0.04,-0.02,0.05,1.00,0.04,0.04
m3 a,-0.05,0.03,0.01,0.02,-0.19,0.04,1.00,0.41
m4 a,0.02,-0.02,0.00,0.01,0.52,0.04,0.41,1.00


In [20]:
train_X, val_X, train_y, val_y = train_test_split(x, y, random_state = 0)

data_model = DecisionTreeRegressor(random_state=0)
data_model.fit(train_X, train_y)

# get predicted prices on validation data
val_predictions = data_model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))

0.4625


In [21]:
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %f" %(max_leaf_nodes, my_mae))

Max leaf nodes: 5  		 Mean Absolute Error:  0.517691
Max leaf nodes: 50  		 Mean Absolute Error:  0.482297
Max leaf nodes: 500  		 Mean Absolute Error:  0.450000
Max leaf nodes: 5000  		 Mean Absolute Error:  0.450000


In [22]:
random_model = RandomForestRegressor(random_state=0)
random_model.fit(train_X, train_y)

val_preds = random_model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))

0.4625
